## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pasvalys,LT,56.06,24.40,67.32,96,64,5.82,broken clouds
1,1,Atuona,PF,-9.80,-139.03,77.83,71,1,17.92,clear sky
2,2,Ribeira Grande,PT,38.52,-28.70,72.01,85,55,8.61,broken clouds
3,3,Talnakh,RU,69.49,88.40,61.65,86,96,4.45,light rain
4,4,New Norfolk,AU,-42.78,147.06,55.92,63,53,5.01,light rain


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
temp_min=float(input("Enter Min Temp"))
temp_max=float(input("Enter Max Temp"))

Enter Min Temp60
Enter Max Temp100


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc meth
Chosen_Temps_df=city_data_df.loc[(city_data_df["Max Temp"]<=temp_max)&(city_data_df["Max Temp"]>=temp_min)]

In [9]:
#4a. Determine if there are any empty rows
Chosen_Temps_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df=Chosen_Temps_df.dropna()

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pasvalys,LT,67.32,broken clouds,56.06,24.40,
1,Atuona,PF,77.83,clear sky,-9.80,-139.03,
2,Ribeira Grande,PT,72.01,broken clouds,38.52,-28.70,
3,Talnakh,RU,61.65,light rain,69.49,88.40,
5,Avarua,CK,77.05,few clouds,-21.21,-159.78,
6,Teguldet,RU,61.92,overcast clouds,57.31,88.17,
7,Lieksa,FI,60.94,broken clouds,63.32,30.02,
8,Ponta Do Sol,PT,70.72,clear sky,32.67,-17.10,
10,Yellowknife,CA,61.29,broken clouds,62.46,-114.35,
11,Hamilton,US,76.35,clear sky,39.18,-84.53,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for i,row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    latitude=row["Lat"]    
    longitude=row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    url_base="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:

        # 6e. Make request and retrieve the JSON data from the search. 
        result_hotels=requests.get(url_base,params=params).json()
    
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[i,"Hotel Name"]=result_hotels["results"][0]["name"]
        
    except:
        pass

In [29]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Pasvalys,LT,67.32,broken clouds,56.06,24.40,
1,Atuona,PF,77.83,clear sky,-9.80,-139.03,
2,Ribeira Grande,PT,72.01,broken clouds,38.52,-28.70,
3,Talnakh,RU,61.65,light rain,69.49,88.40,
5,Avarua,CK,77.05,few clouds,-21.21,-159.78,
...,...,...,...,...,...,...,...
680,Margate,GB,63.03,overcast clouds,51.38,1.39,
681,Inzhavino,RU,65.55,scattered clouds,52.32,42.49,
682,Golden,US,86.36,few clouds,39.76,-105.22,
684,Moses Lake,US,79.48,clear sky,47.13,-119.28,


In [30]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]


In [24]:
hotel_info

['\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Pasvalys</dd>\n<dt>Country</dt><dd>LT</dd>\n<dt>Current Description</dt><dd>broken clouds</dd>\n<dt>Max Temp</dt><dd>67.32</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Description</dt><dd>clear sky</dd>\n<dt>Max Temp</dt><dd>77.83</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Ribeira Grande</dd>\n<dt>Country</dt><dd>PT</dd>\n<dt>Current Description</dt><dd>broken clouds</dd>\n<dt>Max Temp</dt><dd>72.01</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Talnakh</dd>\n<dt>Country</dt><dd>RU</dd>\n<dt>Current Description</dt><dd>light rain</dd>\n<dt>Max Temp</dt><dd>61.65</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd></dd>\n<dt>City</dt><dd>Avarua</dd>\n<dt>Country</dt><dd>CK</dd>\n<dt>Current Description</dt><dd>few clouds</dd>\n<dt>Max Temp</dt><dd>77.05</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt>

In [25]:
locations

,Lat,Lng
0,56.06,24.40
1,-9.80,-139.03
2,38.52,-28.70
3,69.49,88.40
5,-21.21,-159.78
...,...,...
680,51.38,1.39
681,52.32,42.49
682,39.76,-105.22
684,47.13,-119.28


In [31]:
# 11a. Add a marker layer for each city to the map. 
fig=gmaps.figure()
marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))